# Get all topics from GEMET in all languages
* save it to a csv file
* gemet.rdf downloaded from https://www.eionet.europa.eu/gemet/en/exports/rdf/latest

In [8]:
from rdflib import Graph, URIRef
from pprint import pprint
#import rdflib
#import six
import time

from pympler import asizeof
import pandas as pd

In [4]:
# load gemet.rdf
g = Graph()

rdf_file = '/home/dzon/kajo/topics/gemet.rdf'

start = time.time()
g = g.parse(rdf_file)
print('Loading RDF file %.4f s' % ((time.time()-start)))

asizeof.asizeof(g)

Loading RDF file 25.7971 s


263687800

In [6]:
# explore gemet langugages and relations
lang_set = set()
p_set = set()
i = 0
for s,p,o in g:
    i += 1
    p_set.add(p)
    
    try:
        #print(o.language)
        lang_set.add(o.language)
        #print(s,p,o)
    except:
        pass
    
print('PREDICATES count: ', len(p_set))
for keys in p_set:
    print(keys)
    
print('\nnumber fo LANGUAGES: ', len(lang_set))
for keys in lang_set:
    print(keys)  

PREDICATES count:  34
http://xmlns.com/foaf/0.1/page
http://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#subGroupOf
http://www.w3.org/2004/02/skos/core#notation
http://www.w3.org/2004/02/skos/core#exactMatch
http://www.w3.org/2004/02/skos/core#relatedMatch
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://purl.org/dc/terms/title
http://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#sameEEAGlossary
http://purl.org/dc/terms/created
http://www.w3.org/2004/02/skos/core#altLabel
http://www.w3.org/2004/02/skos/core#editorialNote
http://www.w3.org/2004/02/skos/core#member
http://www.w3.org/2004/02/skos/core#definition
http://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#acronymLabel
http://www.w3.org/2004/02/skos/core#related
http://purl.org/dc/terms/creator
http://www.w3.org/2004/02/skos/core#inScheme
http://www.w3.org/2004/02/skos/core#broader
http://www.w3.org/2004/02/skos/core#hiddenLabel
http://www.w3.org/2004/02/skos/core#narrowMatch
http://www.w3.org/2004/02/skos

In [7]:
# count the topics for each language and save the topic names for every language
lang_count = dict()
lang_terms = dict()
for lang in lang_set:
    literal_list_en = set()
    lang_count[lang] = 0
    
    total_count = 0


    for s, o in g.subject_objects(URIRef("http://www.w3.org/2004/02/skos/core#prefLabel")):
        total_count += 1
        try:
            if o.language == lang:
                #print(s,o)
                if lang in lang_count: 
                    lang_count[lang] += 1
                else:
                    lang_count[lang] = 1
                if s in lang_terms:
                    
                    lang_terms[s][lang] = o
                else:
                    lang_terms[s] = dict()
                    lang_terms[s][lang] = o
        except:
            continue

    print(lang, lang_count[lang]) 

ka 5255
es 5344
lv 5344
mt 5344
tr 5244
ga 5344
is 5206
hr 4866
ro 5344
lt 5344
zh-CN 5208
pl 5344
sk 5278
nl 5344
cs 5242
sv 4925
None 0
sl 5344
uk 5244
en-US 5244
da 5327
hu 5343
de 5344
hy 5255
no 5244
az 5255
ar 5244
ru 5244
et 5344
fi 5341
bg 5344
fr 5343
it 5344
en 5565
el 5340
pt 5344
eu 5244
ca 5244


In [10]:
# prepare pandas header
list_lang_keys = list(lang_count.keys())
list_lang_keys.remove(None)
list_lang_keys = sorted(list_lang_keys)
list_lang_keys = ["id", "link"] + list_lang_keys

In [12]:
# transform the language-topic dictionaries to dataframe
import pandas as pd
df = pd.DataFrame(columns=list_lang_keys)
                               
rows_list = []
for row in sorted(lang_terms.keys()):
    #del lang_terms[row][id]
    #del lang_terms[row]["idd"]
    row_parts = row.split("/")
    lang_terms[row]["id"] = row_parts[-1]
    lang_terms[row]["link"] = row
    rows_list.append(lang_terms[row])

df = pd.DataFrame(rows_list)    
df = df[list_lang_keys]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 39 columns):
id       5565 non-null object
link     5565 non-null object
ar       5244 non-null object
az       5255 non-null object
bg       5344 non-null object
ca       5244 non-null object
cs       5242 non-null object
da       5327 non-null object
de       5344 non-null object
el       5340 non-null object
en       5565 non-null object
en-US    5244 non-null object
es       5344 non-null object
et       5344 non-null object
eu       5244 non-null object
fi       5341 non-null object
fr       5343 non-null object
ga       5344 non-null object
hr       4866 non-null object
hu       5343 non-null object
hy       5255 non-null object
is       5206 non-null object
it       5344 non-null object
ka       5255 non-null object
lt       5344 non-null object
lv       5344 non-null object
mt       5344 non-null object
nl       5344 non-null object
no       5244 non-null object
pl       5344 non-null 

In [13]:
df.head()

,id,link,ar,az,bg,ca,cs,da,de,el,...,pl,pt,ro,ru,sk,sl,sv,tr,uk,zh-CN
0,100,http://www.eionet.europa.eu/gemet/concept/100,هيئة إدارية,inzibati qurum,Административен орган,òrgans administratius,orgán správní,administrativt organ,Verwaltungsbehörde,διοικητικός φορέας,...,organ administracji,corpos administrativos,organism administrativ,административный орган,správny úrad,upravni organ,myndighet,"idari kurum, kuruluş",адміністративний орган,行政体
1,10002,http://www.eionet.europa.eu/gemet/concept/10002,المحاسبة,mühasibat uçotu,Счетоводство,comptabilitat,účetnictví,regnskab,Buchführung,λογιστική,...,księgowość,contabilidade,contabilitate,бухгалтерский учет,účtovná evidencia,računovodstvo,bokföring,muhasebe,бухгалтерський облік,会计学
2,10003,http://www.eionet.europa.eu/gemet/concept/10003,حياةالحيوان,heyvanların həyatı,Животински свят,vida animal,život zvířat,dyreliv,Tierleben,πανίδα,...,życie zwierząt,vida animal,viață a animalelor,жизнь животных,NaN,življenje živali,NaN,hayvan yaşamı,життя тварин,动物生命
3,10008,http://www.eionet.europa.eu/gemet/concept/10008,منتج استهلاكي,istehlak məhsulu,Потребителски продукт,producte de consum,výrobek spotřební,forbrugsvare,Konsumprodukt,καταναλωτικό προϊόν,...,produkt konsumpcyjny,produtos de consumo,produs al consumatorului,потребительский продукт,spotrebný výrobok,potrošniški izdelek,konsumentvara,tüketim ürünleri,споживчий продукт,消费品
4,1001,http://www.eionet.europa.eu/gemet/concept/1001,جسر,körpü,Мост,pont,most,bro,Brücke,γέφυρα,...,most,pontes,pod,мост,most,most,bro,köprü,міст,桥


In [ ]:
# save the topics to csv file
df.to_csv('/home/dzon/kajo/topics/gemet_topics_lang.csv', sep = ";")